In [ ]:
%%capture
from __future__ import print_function
import os
import ipywidgets as ipyw
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
import IPython
import warnings
warnings.filterwarnings("ignore")
import mjknowling

## MyVineyard

## MyManagementScenarios

In [ ]:
%matplotlib inline
irrig_scen = ["low", "high"]
irrig_str = ipyw.Text("0.5")
str_d = {"irrigation": irrig_str}
def set_irrig(**kwargs):
    for k, v in kwargs.items():
        str_d[k].value = str(v)
w = ipyw.interactive(set_irrig, irrigation=irrig_scen)
#display(w)
str_d['irrigation'].value = 'low'

## MyAssessment

In [ ]:
plot_types = ["irrigationtimeseries", "irrigationcost", "harvestyield", "harvestrevenue",
              "laitimeseries", "costcontributions", "grossmargin"]
plot_str = ipyw.Text("0.5")
p_d = {"plot": plot_str}
def set_plot_type(**kwargs):
    for k, v in kwargs.items():
        p_d[k].value = str(v)
p = ipyw.interactive(set_plot_type, plot=plot_types)
display(p)

In [ ]:
from IPython.display import clear_output
def on_button_clicked(b):
    clear_output()
    display(button)
    irrig = str_d['irrigation'].value
    #print("evaluating {} irrigation scenario with respect to base scenario...".format(irrig))
    scens = mjknowling.run_scen(irrig)
    plot = p_d['plot'].value
    #print("plotting {}".format(plot))
    mjknowling.plot_scen(scens, plot)
    
button = ipyw.Button(description="Get Some Science")
button.on_click(on_button_clicked)
display(button)

In [ ]:
from ipywidgets import Dropdown
d0 = {'where my water is going' : ['irrigation', 'soil moisture', 'transpiration'], 
      'how my vine is developing' : ['key dates', 'growth', 'end of season'], 
      'what my financial position is': ['revenue', 'costs', 'bottom line']}
d1 = {v: [] for v in [x for y in list(d0.values()) for x in y]}  # one level at a time - to avoid multi-index hassle
d1['irrigation'] = ['time series', 'total']
d1['key dates'] = ['bud burst', 'first flower', 'veraison', 'harvest']
d1['growth'] = ['canopy', 'berry']
d1['end of season'] = ['yield', 'Brix']
d1['revenue'] = ['crush']
d1['costs'] = ['irrigation', 'labour', 'diesel', 'chemical', 'comparison']
d1['bottom line'] = ['gross margin']

In [ ]:
level0_widget = Dropdown( options=list(d0.keys()),
                         value='where my water is going',
                         description=' ',
                       )
level1_widget = Dropdown( options=d0['where my water is going'],
                         value=None,
                         description=' ',
                       )
level2_widget = Dropdown( options=d0['where my water is going'],
                         value=None,
                         description=' ',
                       )

In [ ]:
def on_update_widget_level0(*args):
    level1_widget.options = d0[level0_widget.value]
level0_widget.observe(on_update_widget_level0, 'value')

def on_update_widget_level1(*args):
    level2_widget.options = d1[level1_widget.value]
level1_widget.observe(on_update_widget_level1, 'value')

In [ ]:
print("I want to know...")
display(level0_widget)
display(level1_widget)
display(level2_widget)

In [ ]:
def plot_request_mapper():
    plot_str = ipyw.Text("0.5")
    p_d = {"plot": plot_str}
    if level1_widget.value == "irrigation" and level2_widget.value == "time series":
        p_d['plot'].value = 'irrigationtimeseries'
    elif level1_widget.value == "costs" and level2_widget.value == "irrigation":
        p_d['plot'].value = 'irrigationcost'
    elif level1_widget.value == "end of season" and level2_widget.value == "yield":
        p_d['plot'].value = 'harvestyield'
    elif level1_widget.value == "revenue" and level2_widget.value == "crush":
        p_d['plot'].value = 'harvestrevenue'
    elif level1_widget.value == "growth" and level2_widget.value == "canopy":
        p_d['plot'].value = 'laitimeseries'
    elif level1_widget.value == "costs" and level2_widget.value == "comparison":
        p_d['plot'].value = 'costcontributions'
    elif level1_widget.value == "bottom line" and level2_widget.value == "gross margin":
        p_d['plot'].value = 'grossmargin'
    else:
        p_d['plot'].value = 'underdev'
    return p_d

In [ ]:
level0_widget.value, level1_widget.value, level2_widget.value

In [ ]:
from IPython.display import clear_output
def on_button_clicked(b):
    clear_output()
    display(button)
    irrig = str_d['irrigation'].value
    #print("evaluating {} irrigation scenario with respect to base scenario...".format(irrig))
    scens = mjknowling.run_scen(irrig)  # not wrt baseline
    p_d = plot_request_mapper()
    plot = p_d['plot'].value
    #print("plotting {}".format(plot))
    mjknowling.plot_scen(scens, plot)
    
button = ipyw.Button(description="Get Some Science")
button.on_click(on_button_clicked)
display(button)